In [ ]:
!pip install openpyxl
!pip install pandas

In [ ]:
import numpy as np
import openpyxl
import pandas as pd
from pathlib import Path
import pdfplumber
import re

RECIPES_DIR = "./data"

#
# Read pdf and convert to text.
#
def pdf2text(pdffile):
    all_text = ""
    with pdfplumber.open(pdffile) as pdf:
        for page in pdf.pages:
            # Extract text from the page
            text = page.extract_text()
            all_text += page.extract_text() + "\n"
            
    print(all_text)  # Print the extracted text
    return all_text


# 
# Convert text to a dataframe.
#
def text2dataframe(all_text):
    # find interesting lines
    # Name Well Amount Unit
    # 2-propanol 100% v/v B10 360 ul
    
    # the main pattern to find interesting rows
    pattern = re.compile(r'(\S[\S\s]*?\S)\s+([A-Z][0-9]{1,2})\s+([0-9]*)\s+(.*)$')
    
    # patterns to pick pH and concentration
    pattern_ph = re.compile(r'(\d+\.?\d*)pH')
    pattern_mol = re.compile(r'(\d+\.?\d*) M')
    pattern_vv = re.compile(r'(\d+\.?\d*)% v/v')
    pattern_wv = re.compile(r'(\d+\.?\d*)% w/v')
    
    parsed = []
    lines = all_text.split('\n')
    for line in lines:
        match = pattern.match(line)
        if match:
            name = match.groups()[0]
            well = match.groups()[1]
            amount = match.groups()[2]
            unit = match.groups()[3]
    
            ph = None
            mol = None
            vv = None
            wv = None
    
            match = pattern_ph.search(name)
            if match:
                ph = match.groups()[0]
    
            match = pattern_mol.search(name)
            if match:
                mol = match.groups()[0]
    
            match = pattern_vv.search(name)
            if match:
                vv = match.groups()[0]
    
            match = pattern_wv.search(name)
            if match:
                wv = match.groups()[0]
    
            parsed.append((well, name, ph, mol, vv, wv, amount, unit))
    
    # Function to create a sorting key
    def natural_sort_key(s):
        match = re.match(r"([A-Z]+)([0-9]+)", s)
        if match:
            return match.group(1), int(match.group(2))
        return s, 0
    
    df = pd.DataFrame(parsed, columns=['well','name','pH', 'M', 'vv', 'wv','volume', 'unit'])
    df = df.sort_values(by=['well','name'], key=lambda x: x.map(natural_sort_key))
    df.pH = df.pH.astype(float)
    df.M = df.M.astype(float)
    df.vv = df.vv.astype(float)
    df.wv = df.wv.astype(float)
    df.volume = df.volume.astype(float)
    
    # Group by well and calculate the total volume
    total_volumes = df.groupby('well')['volume'].sum().reset_index()
    total_volumes.rename(columns={'volume': 'total_volume'}, inplace=True)
    print("total volumes (unique):")
    print(total_volumes.total_volume.unique())
    
    # Merge the total volumes back into the original DataFrame
    df = df.merge(total_volumes, on='well')
    
    ## Convert pH to hydrogen ion concentration
    #df['H_concentration'] = 10 ** (-df['pH'])
    
    ## Calculate weighted hydrogen ion concentration for each well
    #df['weighted_H_concentration'] = df['H_concentration'] * df['volume']
    
    ## Sum weighted concentrations and total volumes by well
    #weighted_H_sum = df.groupby('well')['weighted_H_concentration'].sum().reset_index()
    #weighted_H_sum.rename(columns={'weighted_H_concentration': 'total_weighted_H_concentration'}, inplace=True)
    
    ## Merge the total volumes and weighted hydrogen concentrations back into the original DataFrame
    #df = df.merge(weighted_H_sum, on='well')
    
    ## Calculate the average hydrogen ion concentration and convert it back to pH
    #df['average_H_concentration'] = df['total_weighted_H_concentration'] / df['total_volume']
    #df['average_pH'] = -np.log10(df['average_H_concentration'])
    
    return df


# 
# Make xlsx nicer to read.
#
def adjust_col_width(xlsx):
    wb = openpyxl.load_workbook(xlsx)
    worksheet = wb['Sheet1']
    
    worksheet.column_dimensions['A'].width = 5   # well
    worksheet.column_dimensions['B'].width = 40  # name
    worksheet.column_dimensions['C'].width = 5   # pH
    worksheet.column_dimensions['D'].width = 5   # M
    worksheet.column_dimensions['E'].width = 5   # vv
    worksheet.column_dimensions['F'].width = 5  # wv
    worksheet.column_dimensions['G'].width = 10   # volume
    worksheet.column_dimensions['H'].width = 5  # unit
    worksheet.column_dimensions['I'].width = 15  # total volume

    worksheet.column_dimensions['N'].width = 15  # average pH
    for cell in worksheet['N']:
        cell.number_format = '#0.000'

    wb.save(filename=xlsx)


# 
# Process all pdfs in the recipe folder
#

data = Path(RECIPES_DIR)
pdfs = data.glob('*.pdf')

for pdffile in pdfs:
    print(pdffile)
    
    text = pdf2text(pdffile)
    df = text2dataframe(text)

    xlsx = data / (pdffile.stem + '.xlsx')
    df.to_excel(xlsx, index=False)
    print(xlsx)
    adjust_col_width(xlsx)
